# Preprocessing

## Read data

In [ ]:
import pandas as pd
import numpy as np

file_name = "s_2018_all_raw.xlsx"
data = pd.read_excel(file_name)
data

## remove comments at the end of the assignment

In [ ]:
# Taken from NB
lectures_due_dates = {'bio2018_lecture01':'2018-04-09',
 'bio2018_lecture02':'2018-04-09',
 'bio2018_lecture03':'2018-04-09',
 'bio2018_lecture04':'2018-04-09',
 'bio2018_lecture05':'2018-04-11',
 'bio2018_lecture06':'2018-04-13',
 'bio2018_lecture07':'2018-04-16',
 'bio2018_lecture08':'2018-04-18',
 'bio2018_lecture09':'2018-04-20',
 'bio2018_lecture10':'2018-04-23',
 'bio2018_lecture11':'2018-04-25',
 'bio2018_lecture12':'2018-04-30',
 'bio2018_lecture13':'2018-05-02',
 'bio2018_lecture14':'2018-05-04',
 'bio2018_lecture15':'2018-05-07',
 'bio2018_lecture16':'2018-05-09',
 'bio2018_lecture17':'2018-05-11',
 'bio2018_lecture18':'2018-05-18',
 'bio2018_lecture20':'2018-05-21',
 'bio2018_lecture21':'2018-05-23',
 'bio2018_lecture22':'2018-05-25',
 'bio2018_lecture23':'2018-05-30',
 'bio2018_lecture24':'2018-06-01',
 'bio2018_lecture25':'2018-06-04',
 'bio2018_lecture26':'2018-06-06'}


lectures_due_hours = {'bio2018_lecture01':10,
 'bio2018_lecture02':10,
 'bio2018_lecture03':10,
 'bio2018_lecture04':22,
 'bio2018_lecture05':22,
 'bio2018_lecture06':22,
 'bio2018_lecture07':22,
 'bio2018_lecture08':22,
 'bio2018_lecture09':22,
 'bio2018_lecture10':22,
 'bio2018_lecture11':22,
 'bio2018_lecture12':22,
 'bio2018_lecture13':22,
 'bio2018_lecture14':22,
 'bio2018_lecture15':22,
 'bio2018_lecture16':22,
 'bio2018_lecture17':22,
 'bio2018_lecture18':22,
 'bio2018_lecture20':22,
 'bio2018_lecture21':22,
 'bio2018_lecture22':22,
 'bio2018_lecture23':22,
 'bio2018_lecture24':22,
 'bio2018_lecture25':22,
 'bio2018_lecture26':22}
 



# Get date and hour columns
data["date"] = data["ctime"].str.split("T").str[0]
data["ctime"] = data["ctime"].astype("datetime64[ns]")
data["hour"] = data["ctime"].dt.hour


data_temp = data.copy()
df_list = []
# Filter comments too close to due date (close=written 5 hours or less before due date)
for lec,date in lectures_due_dates.items():
    df_list.extend(data_temp[((data_temp["Source_lecture"]==lec) & (data_temp["date"]!=date)) | 
                                 ((data_temp["Source_lecture"]==lec) & (data_temp["date"]==date) & (data_temp["hour"]<lectures_due_hours[lec]-5)) |
                            ((data_temp["Source_lecture"]==lec) & (data_temp["date"]==date) & (data_temp["hour"]>=lectures_due_hours[lec]-5) & (pd.notna(data_temp["parent_id"])))].values.tolist())
    
data = pd.DataFrame(df_list, columns = list(data))
data.head(5)
    

## Remove teacher authors

In [ ]:
teachers_ids = ["310455","996299", "1088779","1089355"]
data = data[~data['author_id'].isin(teachers_ids)]



## Get aggregated data by student

In [ ]:
authors = set(data["author_id"].unique().tolist())
lectures = set(data["Source_lecture"].unique().tolist())

all_comments_dict = data.groupby(["author_id","Source_lecture"])["text"].count().to_dict()

first_comments_dict = data[data["parent_id"].isna()].groupby(["author_id","Source_lecture"])["text"].count().to_dict()

reply_comments_dict = data[~data["parent_id"].isna()].groupby(["author_id","Source_lecture"])["text"].count().to_dict()

# append zeros when missing
for key in all_comments_dict:
    if key not in first_comments_dict.keys():
        first_comments_dict[key] = 0
    if key not in reply_comments_dict.keys():
        reply_comments_dict[key] = 0

In [ ]:
print(f"Number of comments: {sum(all_comments_dict.values())}")
print(f"Number of Students: {len(authors)}") 

## Remove students that didn't comment over all of the lectures

In [ ]:
print("Number of students before")
print(len(all_comments_dict.keys()))
print(len(first_comments_dict.keys()))
print(len(reply_comments_dict.keys()))

In [ ]:
lectures = sorted(list(set(data["Source_lecture"].unique().tolist())))
lectures_per_student = data.groupby(["author_id"])["Source_lecture"].nunique().to_dict()
all_comments_keys = list(all_comments_dict.keys())
first_comments_keys = list(first_comments_dict.keys())
reply_comments_keys = list(reply_comments_dict.keys())

for k in all_comments_keys:
    if lectures_per_student[k[0]] != 25:
        del all_comments_dict[k]
        if k in first_comments_keys:
            del first_comments_dict[k]
        if k in reply_comments_keys:
            del reply_comments_dict[k]
        



In [ ]:
print("Number of students after")
print(len(all_comments_dict.keys()))
print(len(first_comments_dict.keys()))
print(len(reply_comments_dict.keys()))

## Get number of comments in the next lecture

In [ ]:
all_keys = sorted(list(all_comments_dict.keys()))
good_authors = list(set(k[0] for k in all_keys))

next_all_comments_dict = {}
for curr_l, next_l in zip(lectures, lectures[1:]):
    for author in good_authors:
        next_all_comments_dict[(author,curr_l)] = all_comments_dict[(author,next_l)]
        if next_l == "bio2018_lecture26":
            next_all_comments_dict[(author,next_l)] = -1

In [ ]:
print(f"Number of Comments: {sum(all_comments_dict.values())}")
print(f"Number of Students: {len(good_authors)}") 

## Get treatment label
Treatment = At least one of the student's posts got answered

In [ ]:
thread_length_dict = data.groupby(["location_id"])["text"].count().to_dict()
data["got_answered"] = data.apply(lambda row: 1 if (thread_length_dict[row["location_id"]]>1 and pd.isnull(row["parent_id"])) else 0, axis=1)
got_answered_dict = data.groupby(["author_id","Source_lecture"])["got_answered"].sum().to_dict()

# Change to binary value (1=Got answered, 0=Not answered)
for k,v in got_answered_dict.items():
    if v>0:
        got_answered_dict[k] = 1

## Get verbosity level

In [ ]:
data["text"] = data["text"].astype(str)
student_lecture_verbosity = data.groupby(["author_id","Source_lecture"])["text"].apply(lambda x: "\n".join(x)).apply(lambda x: len(x))

## Get emoji counts

In [ ]:
confused_hashtag = data.groupby(["author_id","Source_lecture"])["#confused"].sum().to_dict()
curious_hashtag = data.groupby(["author_id","Source_lecture"])["#curious"].sum().to_dict()
help_hashtag = data.groupby(["author_id","Source_lecture"])["#help"].sum().to_dict()
useful_hashtag = data.groupby(["author_id","Source_lecture"])["#useful"].sum().to_dict()
interested_hashtag = data.groupby(["author_id","Source_lecture"])["#interested"].sum().to_dict()
frustrated_hashtag = data.groupby(["author_id","Source_lecture"])["#frustrated"].sum().to_dict()
question_hashtag = data.groupby(["author_id","Source_lecture"])["#question"].sum().to_dict()
idea_hashtag = data.groupby(["author_id","Source_lecture"])["#idea"].sum().to_dict()

## Get section id

In [ ]:
student_lecture_section_id = data.groupby(["author_id","Source_lecture"])["section_id"].max()

# Correct for students with NAN value in section id
studet_section_id = {k[0]:[] for k in student_lecture_section_id.keys()}
for k,v in student_lecture_section_id.items():
    studet_section_id[k[0]].append(v)
    

for k,v in student_lecture_section_id.items():
    if pd.isna(v):
        student_lecture_section_id[k] = max(studet_section_id[k[0]])
        

# weird student, this is a manual correction
student_lecture_section_id[(1121374, 'bio2018_lecture01')] = 0
student_lecture_section_id[(1121374, 'bio2018_lecture02')] = 0
student_lecture_section_id[(1121374, 'bio2018_lecture03')] = 0
student_lecture_section_id[(1121374, 'bio2018_lecture018')] = 0

## Create final dataframe
Remove lecture 26 - This is the last lecture and doesn't contain Y value(number of comments in the next lecture).

In [ ]:
df_list = []
for k,v in all_comments_dict.items():
    # Remove lecture 26
    if k[1]=="bio2018_lecture26":
        continue
    curr_list = []
    curr_list.append(k[0]) # author
    curr_list.append(k[1]) # lecture
    curr_list.append(v) # Number of comments in current lecture

    curr_list.append(first_comments_dict[k]) # Number of first-comments
    curr_list.append(reply_comments_dict[k]) # Number of reply-comments
    curr_list.append(student_lecture_verbosity[k]) # Student verbosity
    curr_list.append(student_lecture_section_id[k]) # Student section id
    
    curr_list.append(confused_hashtag[k])   
    curr_list.append(curious_hashtag[k])   
    curr_list.append(help_hashtag[k]) 
    curr_list.append(useful_hashtag[k])
    curr_list.append(interested_hashtag[k])
    curr_list.append(frustrated_hashtag[k])
    curr_list.append(question_hashtag[k]) 
    curr_list.append(idea_hashtag[k]) 

    
    
    curr_list.append(got_answered_dict[k]) # Is got answered in the current lecture (T)
    curr_list.append(next_all_comments_dict[k]) # Number of coments ins the next lecture (Y)
    
    df_list.append(curr_list)
final_df = pd.DataFrame(df_list, columns=["author","lecture","#total-comments","#first-comments","#reply-comments",
                                          "verbosity","section_id","confused_hashtag", "curious_hashtag", 
                                          "help_hashtag", "useful_hashtag", "interested_hashtag", 
                                          "frustrated_hashtag", "question_hashtag", "idea_hashtag", "T","Y"])
final_df 

## Create dataframe for causal model

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale numerical values
features_df = final_df[["#total-comments","#first-comments","#reply-comments","verbosity", "confused_hashtag", "curious_hashtag", "help_hashtag", "useful_hashtag", "interested_hashtag", "frustrated_hashtag", "question_hashtag", "idea_hashtag"]]
scaler = StandardScaler()
features_df_scaled = pd.DataFrame(scaler.fit_transform(features_df))

# Get one hot encoding of lectures and authors
lectures_df = pd.get_dummies(final_df["lecture"])
authors_df = pd.get_dummies(final_df["author"])
section_df = pd.get_dummies(final_df["section_id"])

# Concat all to a signle dataframe
features_df = pd.concat([features_df_scaled, lectures_df,authors_df,section_df], axis=1)    
features_df

## Number of treated authors (at least once across all lectures)

In [ ]:

authors_got_answered = 0
for k,v in final_df.groupby("author")["T"].sum().to_dict().items():
    if v>0:
        authors_got_answered+=1
print("Number of authors: ", len(authors))
print("Number of authors that got answerd at least once during the semester: ", authors_got_answered)

## TODO:
- Add more features from Davis: Age, sex, demographic, First language

# Calculate Causality

## Check a simple correlation

In [ ]:
final_df[final_df["T"]==0].shape, final_df[final_df["T"]==1].shape

In [ ]:
print(final_df[final_df["T"]==1].groupby("lecture")["Y"].mean().mean())
print(final_df[final_df["T"]==1].groupby("lecture")["Y"].mean().std())

In [ ]:
print(final_df[final_df["T"]==0].groupby("lecture")["Y"].mean().mean())
print(final_df[final_df["T"]==0].groupby("lecture")["Y"].mean().std())

## Check causation

### Get baseline model 

In [ ]:
from causalinference import CausalModel

cm = CausalModel(
    Y=final_df["Y"].values, 
    D=final_df["T"].values, 
    X=features_df.values)

cm.est_via_matching()
# cm.est_via_ols()

print(cm.estimates)

In [ ]:
features_df.shape

In [ ]:
from causalinference import CausalModel

cm = CausalModel(
    Y=final_df["Y"].values, 
    D=final_df["T"].values, 
    X=features_df.values)

# cm.est_propensity_s()
cm.est_via_matching()
# cm.est_via_ols()

print(cm.estimates)

# Groupby lecture

In [ ]:
wanted_features = [feature for feature in list(features_df) if not str(feature).startswith("bio2018")]
wanted_features

In [ ]:
from causalinference import CausalModel

prev_data_D_Y = None
prev_data_features = None
lectures = sorted(final_df["lecture"].unique().tolist())
results2 = []
for lecture in lectures:
    print(f"Lecture: {lecture}")
    
    if prev_data_D_Y is not None:
        prev_data_D_Y = pd.concat([prev_data_D_Y, final_df[final_df["lecture"]==lecture]], axis=0)
        prev_data_features = pd.concat([prev_data_features, features_df[features_df[lecture]==1][wanted_features]],axis=0)
        
    else:
        prev_data_D_Y = final_df[final_df["lecture"]==lecture]
        prev_data_features = features_df[features_df[lecture]==1][wanted_features]
        
    print(prev_data_features.shape)
    if lecture=="bio2018_lecture01" or lecture=="bio2018_lecture02":
        continue
    cm = CausalModel(
        Y=prev_data_D_Y["Y"].values, 
        D=prev_data_D_Y["T"].values, 
        X=prev_data_features.values)

    cm.est_via_matching()
#     cm.est_propensity_s()
#     cm.est_via_ols()


    print(cm.estimates)
#     results2.append(["OLS", lecture, cm.estimates["ols"]["att"], cm.estimates["ols"]["att_se"]])
    results2.append(["Matching",lecture, cm.estimates["matching"]["att"], cm.estimates["matching"]["att_se"]])

#     print(cm.estimates["matching"]["att"], cm.estimates["matching"]["att_se"])
#     print()

In [ ]:
pd.DataFrame(results2, columns=["Method","Lecture","ATT","S.e"]).to_csv("Lectures_results_matching.csv")

### TODO:
  
 - Calc propensity score
 
 - Calc IPSW
 
 - Calc S-learner
 
 - Calc T-learner

### Get S-Learner

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import feature_selection
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, average_precision_score
from sklearn import feature_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut

est = LinearRegression()
pipeline = Pipeline(
        [
#          ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])



In [ ]:
random_seed = 1234
outer_folds = 10
outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=random_seed)

# Prepare data
X = pd.concat([features_df, final_df["T"]], axis=1)
y = final_df["Y"]
wanted_columns = list(X)
X["S_learner_0"] = 0
X["S_learner_1"] = 0

# Fit
pipeline.fit(X,y)

# Predict
X["T"] = 0
y_pred_0 = pipeline.predict(X)
X["T"] = 1
y_pred_1 = pipeline.predict(X)

# Save results
X["S_learner_0"] = y_pred_0
X["S_learner_1"] = y_pred_1

In [ ]:
data1_copy = final_df.copy()
data1_copy["S_learner_0"] = X["S_learner_0"]
data1_copy["S_learner_1"] = X["S_learner_1"]
ATT_data1_S_learner = (data1_copy[data1_copy["T"]==1]["S_learner_1"] - data1_copy[data1_copy["T"]==1]["S_learner_0"]).mean()
ATT_data1_S_learner_std = (data1_copy[data1_copy["T"]==1]["S_learner_1"] - data1_copy[data1_copy["T"]==1]["S_learner_0"]).std()
print("ATT + std")
ATT_data1_S_learner, ATT_data1_S_learner_std / data1_copy[data1_copy["T"]==1].shape[0]**0.5

In [ ]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m, 3), round(m-h, 3), round(m+h, 3)
print("ATT + Confidence intervals:")
mean_confidence_interval((data1_copy[data1_copy["T"]==1]["S_learner_1"] - data1_copy[data1_copy["T"]==1]["S_learner_0"]))

### Get T-Learner

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer

est = Lasso(alpha=0.1)
# scaler = QuantileTransformer(output_distribution='uniform')
# scaler = StandardScaler()
# selector = feature_selection.RFE(est, 15)
pipeline_0 = Pipeline(
        [
#          ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])

est2 = Lasso(alpha=0.1)
# scaler2 = QuantileTransformer(output_distribution='uniform')
# selector2 = feature_selection.RFE(est, 15)
pipeline_1 = Pipeline(
        [
#          ('scale', scaler2),
#          ('select', selector2),
         ('classifier', est2)])


In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, average_precision_score
from sklearn import feature_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut

random_seed = 1234
outer_folds = 10
outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=random_seed)


# Prepare data
X = pd.concat([features_df, final_df["T"]], axis=1)
y = final_df["Y"]
wanted_columns = list(X)
X["S_learner_0"] = 0
X["S_learner_1"] = 0

X_controle = X[X["T"]==0][wanted_columns]
X_treatment = X[X["T"]==1][wanted_columns]




# Fit
pipeline_0 = pipeline_0.fit(X=X_controle, y=y.iloc[X_controle.index])
pipeline_1 = pipeline_1.fit(X=X_treatment, y=y.iloc[X_treatment.index])

# Predict
y_pred_0 = pipeline_0.predict(X[wanted_columns])
y_pred_1 = pipeline_1.predict(X[wanted_columns])
   
    
# Save results
X["S_learner_0"] = y_pred_0
X["S_learner_1"] = y_pred_1    
    


In [ ]:
data1_copy = final_df.copy()
data1_copy["S_learner_0"] = X["S_learner_0"]
data1_copy["S_learner_1"] = X["S_learner_1"]
ATT_data1_S_learner = (data1_copy[data1_copy["T"]==1]["S_learner_1"] - data1_copy[data1_copy["T"]==1]["S_learner_0"]).mean()
ATT_data1_S_learner_std = (data1_copy[data1_copy["T"]==1]["S_learner_1"] - data1_copy[data1_copy["T"]==1]["S_learner_0"]).std()
print("ATT + std")
ATT_data1_S_learner, ATT_data1_S_learner_std/data1_copy[data1_copy["T"]==1].shape[0]**0.5

In [ ]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return f"{round(m, 3)} ({round(m-h, 3)}, {round(m+h, 3)})"
print("ATT + Confidence intervals:")
mean_confidence_interval((data1_copy[data1_copy["T"]==1]["S_learner_1"] - data1_copy[data1_copy["T"]==1]["S_learner_0"]))

### Matching

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR


knn = KNeighborsRegressor(n_neighbors=1000)
# X = pd.concat([features_df, final_df["T"]], axis=1)
# knn = KNeighborsRegressor(n_neighbors=100, algorithm='brute', metric='mahalanobis', metric_params={'V': np.cov(X)})

est = Lasso(alpha=0.1)
# scaler = StandardScaler()

# scaler = QuantileTransformer(output_distribution='uniform')
# selector = feature_selection.RFE(est, 15)
pipeline = Pipeline(
        [
#          ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])



In [ ]:

from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, average_precision_score
from sklearn import feature_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut

random_seed = 1234
outer_folds = 10
outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=random_seed)

# Prepare data
X = pd.concat([features_df, final_df["T"]], axis=1)
y = final_df["Y"]
wanted_columns = list(X)
X["Matched_y"] = 0
X["predicted_y"] = 0

# Fit
pipeline.fit(X=X[wanted_columns], y=y)
knn.fit(X[wanted_columns], y)

# Predict 
X["predicted_y"] = pipeline.predict(X[wanted_columns])
distances, indices = knn.kneighbors(X[wanted_columns])
for i,v in enumerate(list(X.index)):
    print(i)
    curr_T = X.loc[i,"T"]
    neighbors = X.iloc[indices[i]]
    nn = list(neighbors[neighbors["T"] !=curr_T].index)[0]
    X.loc[i,"Matched_y"] = X.loc[i,"predicted_y"]

In [ ]:
data1_copy = final_df.copy()
data1_copy["Matched_y"] = X["Matched_y"]
ATT_data2_T_learner = (data1_copy[data1_copy["T"]==1]["Y"] - data1_copy[data1_copy["T"]==1]["Matched_y"]).mean()
ATT_data1_S_learner_std = (data1_copy[data1_copy["T"]==1]["Y"] - data1_copy[data1_copy["T"]==1]["Matched_y"]).std()
print("ATT + std")
ATT_data2_T_learner, ATT_data1_S_learner_std / data1_copy[data1_copy["T"]==1].shape[0]**0.5

In [ ]:
import numpy as np
import scipy.stats

print("ATT + Confidence intervals:")
mean_confidence_interval((data1_copy[data1_copy["T"]==1]["Y"] - data1_copy[data1_copy["T"]==1]["Matched_y"]))

### Get IPSW

In [ ]:
def get_pr_auc_curve(model, X, y, is_plot_show=True):
    preds = model.predict_proba(X)
    preds_classes = model.predict(X)
    preds = preds[:, 1]

    precision, recall, thresholds = precision_recall_curve(y, preds)
    # calculate F1 score
    f1 = f1_score(y, preds_classes)
    # calculate precision-recall AUC
    auc_score = auc(recall, precision)
    # calculate average precision score
    ap = average_precision_score(y, preds)
    print('f1=%.3f auc=%.3f ap=%.3f' % (f1, auc_score, ap))
    # plot no skill
    if is_plot_show:
        pyplot.plot([0, 1], [0.1, 0.1], '-b', linestyle='--')
        # plot the precision-recall curve for the model
        pyplot.plot(recall, precision, '-b', marker='.', label="PR-curve=%f"%(auc_score))
        pyplot.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=1)



        # show the plot
        pyplot.title("Curves")
        pyplot.show()

    return f1, auc_score

def get_conf_matrix(model, X, y):
    preds = model.predict(X)
    clf_report = classification_report(y, preds, target_names=["0", "1"])
    print(clf_report)
    conf_mat = confusion_matrix(y, preds, labels=[0, 1])
    print(conf_mat)
    print("F1 score:", f1_score(y, preds))
    return conf_mat, clf_report


#### Learn the best model for propensity score

In [ ]:
print(__doc__)

# Author: Alexandre Gramfort <alexandre.gramfort@telecom-paristech.fr>
#         Jan Hendrik Metzen <jhm@informatik.uni-bremen.de>
# License: BSD Style.

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn import feature_selection
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, average_precision_score
from sklearn import feature_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree


# Create dataset of classification task with many redundant and few
# informative features
X = features_df.copy()
y = final_df["T"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=42)


def plot_calibration_curve(est, name, fig_index):
    """Plot calibration curve for est w/o and with calibration. """
    # Calibrated with isotonic calibration
    isotonic = CalibratedClassifierCV(est, cv=2, method='isotonic')

    # Calibrated with sigmoid calibration
    sigmoid = CalibratedClassifierCV(est, cv=2, method='sigmoid')

    # Logistic regression with no calibration as baseline

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    for clf, name in [(est, name),
                      (isotonic, name + ' + Isotonic'),
                      (sigmoid, name + ' + Sigmoid')]:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_test)[:, 1]
        else:  # use decision function
            prob_pos = clf.decision_function(X_test)
            prob_pos = \
                (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

        clf_score = brier_score_loss(y_test, prob_pos, pos_label=y.max())
        print("%s:" % name)
        print("\tBrier: %1.3f" % (clf_score))
        print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))
        print("\tRecall: %1.3f" % recall_score(y_test, y_pred))
        print("\tF1: %1.3f\n" % f1_score(y_test, y_pred))
        get_conf_matrix(clf, X_test, y_test)
        
        fraction_of_positives, mean_predicted_value = \
            calibration_curve(y_test, prob_pos, n_bins=10)

        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % (name, clf_score))

        ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
                 histtype="step", lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")
    ax2.legend(loc="upper center", ncol=2)

    plt.tight_layout()


    
# Plot calibration curve for Gaussian Naive Bayes
est = GaussianNB()
scaler =  StandardScaler()
pipeline = Pipeline(
        [
#            ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])
plot_calibration_curve(pipeline, "Naive Bayes", 1)

est = LinearSVC(max_iter=10000)
# scaler = QuantileTransformer(output_distribution='uniform')
scaler =  StandardScaler()
# selector = feature_selection.RFE(est, 5)
pipeline = Pipeline(
        [
#          ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])
# Plot calibration curve for Linear SVC
plot_calibration_curve(pipeline, "SVC", 2)

est = LogisticRegression(C=1., solver="lbfgs", penalty='none')
scaler =  StandardScaler()
# selector = feature_selection.RFE(est, 10)
pipeline = Pipeline(
        [
#         ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])
# Plot calibration curve for Linear SVC
plot_calibration_curve(pipeline, "LR", 3)

est = tree.DecisionTreeClassifier(max_depth=6, class_weight={0:1, 1:1})
pipeline = Pipeline(
        [
         ('classifier', est)])
# Plot calibration curve for Linear SVC
plot_calibration_curve(pipeline, "DT", 4)

plt.show()

#### Pick the best model for propensity score

In [ ]:
est = LogisticRegression(C=1., solver="lbfgs", penalty='none')
pipeline = Pipeline(
        [
#         ('scale', scaler),
#          ('select', selector),
         ('classifier', est)])
calibrator = CalibratedClassifierCV(pipeline, cv='prefit', method='sigmoid')


#### Predict propensity score ove the dataset

In [ ]:
final_df["T"].value_counts()

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, average_precision_score
from sklearn import feature_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold


random_seed = 1234
outer_folds = 10
# outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=random_seed)
outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=random_seed)

X = features_df.copy()
y = final_df["T"]

pipeline.fit(X,y)
calibrator.fit(X,y)


# calibrator_preds = calibrator.predict_proba(X)[:,1]

# pipeline_preds = pipeline.predict_proba(X)[:,1]

propensity_pipeline = pd.DataFrame(pipeline.predict_proba(X))
propensity_calibrator = pd.DataFrame(calibrator.predict_proba(X))


#### Propensity score evaluation

In [ ]:
print(f"Propensity score from pipeline: {brier_score_loss(y, propensity_pipeline.loc[:,1], pos_label=y.max())}")
print(f"Propensity score from calibrator: {brier_score_loss(y, propensity_calibrator.loc[:,1], pos_label=y.max())}")

#### Calc IPSW

In [ ]:
final_df_copy = final_df.copy()
# lg = LogisticRegression()
lg = LogisticRegression(C=1., solver="lbfgs", penalty='l2')
X = features_df
y = final_df["T"]
lg.fit(X,y)

propensity = lg.predict_proba(X)[:,1]

final_df_copy["propensity1"] = propensity
final_df_copy["propensity2"] = 1/propensity
final_df_copy["propensity3"] = propensity/(1-propensity)

final_df_copy["w_t"] = final_df_copy.apply(lambda row: row["T"]+((1-row["T"])*row["propensity1"])/(1-row["propensity1"]), axis=1).clip(-10,10)
final_df_copy["score"] = final_df_copy["Y"]*final_df_copy["w_t"]

final_df_copy["ips"] = np.where(
    final_df["T"] == 1, 
    1 / propensity,
    1 / (1 - propensity))

final_df_copy["ipsw"] = final_df_copy["Y"] * final_df_copy.ips

# Trip too big inverse propensity scores
print(final_df_copy.shape)
final_df_copy = final_df_copy[final_df_copy["ipsw"]<30]
print(final_df_copy.shape)


ipse = (
      final_df_copy[final_df_copy["T"] == 1]["ipsw"].sum() 
    - final_df_copy[final_df_copy["T"] == 0]["ipsw"].sum()
) / final_df_copy.shape[0]

print("ATE:", ipse)

In [ ]:
# The above approach is similar to:
Y1 = final_df_copy[final_df_copy["T"] == 1]["ipsw"].sum() / final_df_copy.shape[0]
Y0 = final_df_copy[final_df_copy["T"] == 0]["ipsw"].sum() / final_df_copy.shape[0]
print("ATE:", Y1-Y0)

In [ ]:
final_df_copy.apply(lambda x: x["w_t"]*x["Y"],axis=1).mean()

In [ ]:
from IPython.display import Image
# Taken from here: https://onlinelibrary.wiley.com/doi/epdf/10.1002/bimj.201600094
Image(filename='IPW formula.png')

In [ ]:
Y1_a = final_df_copy[final_df_copy["T"] == 1]["Y"].sum() 
Y1_b =  final_df_copy[final_df_copy["T"] == 1]["Y"].shape[0]

Y0_a = (final_df_copy[final_df_copy["T"] == 0]["Y"] * final_df_copy[final_df_copy["T"] == 0]["w_t"]).sum()
Y0_b = (final_df_copy[final_df_copy["T"] == 0]["propensity3"]).sum()
ipsw_ATT = Y1_a/Y1_b - Y0_a/Y0_b
print("ATT:", ipsw_ATT)

In [ ]:
final_df_copy[final_df_copy["T"] == 1].shape[0]

In [ ]:
final_df_copy["att"] = ipsw_ATT
Y1_a = (final_df_copy[final_df_copy["T"] == 1]["Y"] - final_df_copy[final_df_copy["T"] == 1]["att"]).sum()
Y1_b =  final_df_copy[final_df_copy["T"] == 1]["Y"].shape[0]

Y0_a = (final_df_copy[final_df_copy["T"] == 0]["Y"] * final_df_copy[final_df_copy["T"] == 0]["w_t"] - final_df_copy[final_df_copy["T"] == 0]["att"]).sum()
Y0_b = (final_df_copy[final_df_copy["T"] == 0]["propensity3"]).sum()
ipsw_ATT_std = Y1_a/Y1_b - Y0_a/Y0_b
print("ATT_standard_error:", ipsw_ATT_std/final_df_copy.shape[0]**0.5)

In [ ]:
# def mean_confidence_interval2(mean, std,n, confidence=0.95):
#     n = n
#     m, se = mean, std
#     h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
#     return round(m, 3), round(m-h, 3), round(m+h, 3)
# mean_confidence_interval2(ipsw_ATT, ipwt_std, final_df_copy.shape[0])

### TODO:
read this:
http://www.degeneratestate.org/posts/2018/Mar/24/causal-inference-with-python-part-1-potential-outcomes/